In [1]:
import numpy as np
from astropy import coordinates as coo
from astropy import units as u
import healpy as hp
import matplotlib.pyplot as plt
import multiprocessing as mp
from astropy.io import fits
from data_process import weight_nan_mean, load_calculation_data, calculate_jackknife_variance
from sklearn.neighbors import KDTree

In [3]:
Nside_jack = 4
r_p, dat = load_calculation_data('../calculation_data/result_r=3_100_15_quasar_lss_all_CMB_lcut_2048_tree.npy', return_rp=True)
dat_r = load_calculation_data('../calculation_data/result_r=3_100_15_random_quasar_lss_all_CMB_lcut_2048_tree.npy')


with open('label.bin', 'rb') as f:
    pix_q = np.load(f)
    pix_ra = np.load(f)


In [12]:
quasar_cata = np.load('../catalogue/quasar_lss_all.npy')
c = coo.SkyCoord(ra=quasar_cata['ra']*u.degree, dec=quasar_cata['dec']*u.degree)
l = c.galactic.l.to(u.rad).value
b_q = c.galactic.b.to(u.rad).value
pix_q = hp.ang2pix(Nside_jack, theta=np.pi/2-b_q, phi=l)
print('finish loading quasar catalogue')

random_sample = np.load('../catalogue/random_quasar_lss_all.npy')
c = coo.SkyCoord(ra=random_sample['ra']*u.degree, dec=random_sample['dec']*u.degree)
assert (random_sample['w']==1).all()
l = c.galactic.l.to(u.rad).value
b_r = c.galactic.b.to(u.rad).value
pix_ra = hp.ang2pix(Nside_jack, theta=np.pi/2-b_r, phi=l)


print('finish loading random sample')

finish loading quasar catalogue
finish loading random sample


In [30]:
import operator

In [34]:
mid = np.percentile(abs(b_q), 50)           # this is about 50 deg, you can also choose other value.

for suf, op in zip(['low_b', 'high_b'], [operator.lt, operator.ge]):
    index = op(abs(b_q), mid)
    value = dat[0,index]
    weight = dat[1,index]
    pix = pix_q[index]

    index = op(abs(b_r), mid)
    value_r = dat_r[0,index]
    weight_r = dat_r[1,index]
    pix_r = pix_ra[index]
    sampler_sub  = calculate_jackknife_variance(value, weight, pix, value_r, weight_r, pix_r, norm_cov=True)
    np.savetxt(f'./results/quasar_lss_all_{suf}_cmb_lcut_n_2048', np.c_[r_p, sampler_sub['mean'], sampler_sub['std']])